### Prepare Date

In [1]:
#load libraries
import pandas as pd
import numpy as np

# load fama-french 3-factor
ff3 = pd.read_csv('FF3.CSV')

# process data
ff3.iloc[:,1:] = ff3.iloc[:,1:].apply(lambda x: x/100) # adjust to decimle scale
ff3['date'] = pd.to_datetime(ff3['date'],format='%Y%m')
ff3['year'] = pd.DatetimeIndex(ff3['date']).year
ff3['month'] = pd.DatetimeIndex(ff3['date']).month
ff3 = ff3[ff3['year'] >= 1967].reset_index(drop=True)

ff3.head()

,date,Mkt-RF,SMB,HML,RF,year,month
0,1967-01-01,0.0815,0.0832,0.0222,0.0043,1967,1
1,1967-02-01,0.0078,0.0334,-0.0217,0.0036,1967,2
2,1967-03-01,0.0399,0.0163,0.0031,0.0039,1967,3
3,1967-04-01,0.0389,0.0062,-0.0264,0.0032,1967,4
4,1967-05-01,-0.0433,0.0198,0.0080,0.0033,1967,5


In [2]:
# load crsp data
crsp = pd.read_csv('CRSP-all.csv')

# process data
crsp = crsp[crsp['SHRCD'].isin([10,11])] # only common shares
crsp = crsp[crsp['SHRCLS'] == 'A'] # only primary stock per firm
crsp['SICCD'] = pd.to_numeric(crsp['SICCD'], errors='coerce')
crsp = crsp[~crsp['SICCD'].between(6000,6999)] # no financial firms
crsp['RET'] = pd.to_numeric(crsp['RET'], errors='coerce') # str to int
crsp['RET'] = np.where(crsp['RET']<-50, np.nan, crsp['RET']) # handle codes
crsp['date'] = pd.to_datetime(crsp['date'],format='%Y%m%d')
crsp['year'] = pd.DatetimeIndex(crsp['date']).year
crsp['month'] = pd.DatetimeIndex(crsp['date']).month
crsp['PRC'] = crsp['PRC'].map(abs) # adj for non-close values
crsp['market_value'] = ((crsp['PRC']/crsp['CFACPR'])*(crsp['SHROUT']*crsp['CFACSHR'])).shift(1) # get market value and shift to get month past
crsp = crsp.drop(columns=['SHRCD','SHRCLS','PRC','CFACPR','SHROUT','CFACSHR']) # clean columns

crsp.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERMNO,date,SICCD,RET,year,month,market_value
1,10000,1986-01-31,3990.0,NaN,1986,1,NaN
2,10000,1986-02-28,3990.0,-0.257143,1986,2,16100.0
3,10000,1986-03-31,3990.0,0.365385,1986,3,11960.0
4,10000,1986-04-30,3990.0,-0.098592,1986,4,16330.0
5,10000,1986-05-30,3990.0,-0.222656,1986,5,15172.0


In [3]:
# load compustat data w/ link
link = pd.read_csv('CRSP-COMP-link.csv')

# process data
link = link[link['fic'] == 'USA'] # remove non-US firms
link = link[~link['exchg'].isin([0,1,2,3,4,7,20])] # remove non-target exhanges
link['sic_adj'] = np.where(link['sich'].isnull(),link['sic'],link['sich']) # reconcile missing sich
link = link[~link['sic_adj'].between(6000,6999)] # remove finance firms
link['sales_growth'] = np.where(link['LPERMNO'] == link['LPERMNO'].shift(3),link['sale']/link['sale'].shift(3) - 1,np.nan) # add sales growth
link = link.drop(columns=['GVKEY','datadate','sale','conm','indfmt','consol','popsrc','datafmt','curcd','exchg','costat','fic','sich','sic','sic_adj'])

link.head()

,LPERMNO,fyear,sales_growth
0,25881,1970.0,NaN
1,25881,1971.0,NaN
2,25881,1972.0,NaN
3,25881,1973.0,-0.167310
4,25881,1974.0,0.069993


In [4]:
# merge datasets
data = crsp.merge(link,left_on=['PERMNO','year'],right_on=['LPERMNO','fyear'])
data = data.drop(columns=['LPERMNO','fyear'])
data.head()

,PERMNO,date,SICCD,RET,year,month,market_value,sales_growth
0,10000,1986-01-31,3990.0,NaN,1986,1,NaN,NaN
1,10000,1986-02-28,3990.0,-0.257143,1986,2,16100.0,NaN
2,10000,1986-03-31,3990.0,0.365385,1986,3,11960.0,NaN
3,10000,1986-04-30,3990.0,-0.098592,1986,4,16330.0,NaN
4,10000,1986-05-30,3990.0,-0.222656,1986,5,15172.0,NaN


### Define Functions

In [5]:
# define strategy function
def rebalance(year, step5=False):
    
    # step 1: isolation & back check
    port = data[data['year'] == year] # isolate year
    port = port[~port['sales_growth'].isnull()] # remove firms w/o 3yr history
            
    # step 5: remove top 1000 firms by size
    if step5 == True:
        try:
            # not over 1000 stocks each year, annihilates N some years, and breaks qcut
            port = port.sort_values('market_value')[0:-1000] 
            port['decile'] = pd.qcut(port['sales_growth'],10,labels=list(range(1,11))) # add decile labels
        except:
            port = data[data['year'] == year] # isolate year
            port = port[~port['sales_growth'].isnull()] # remove firms w/o 3yr history
            port['decile'] = pd.qcut(port['sales_growth'],10,labels=list(range(1,11))) # add decile labels
            #print(year, 'FAIL')
            
    else:
        # step 2: make into deciles
        port['decile'] = pd.qcut(port['sales_growth'],10,labels=list(range(1,11))) # add decile labels
    
    port = port.sort_values(['decile','PERMNO','month']) # sort for asthetics
    
    # step 3: build portfolios
    decile = pd.DataFrame()
    for d in range(1,11):
        dec = port[port['decile'] == d]
        mrk_mon = dec.groupby(['month'])['market_value'].sum()
        merge = dec.merge(mrk_mon,left_on='month',right_on='month')
        merge['weight'] = merge['market_value_x']/merge['market_value_y']
        merge['w_ret'] = merge['weight']*merge['RET']
        dec = pd.DataFrame(merge.groupby('month')['w_ret'].sum()).reset_index().rename(columns={'w_ret':'return'})
        dec['year'] = year
        dec['bin'] = d
        decile = decile.append(dec)
    
    return decile

In [6]:
# load regression & reporting libraries
from statsmodels.api import OLS, add_constant

# define testing function
def tests(decile,df):

    # step 4: test agains fama-franch (t & F-tests)
    port = df[df['bin'] == decile]
    port = port.merge(ff3,left_on=['year','month'],right_on=['year','month']).drop(columns=['date'])
    port['rb-rf'] = port['return'] - port['RF'] # get risk premium
    
    # run regression
    X = port[['Mkt-RF','SMB','HML']]
    y = port['rb-rf']
    
    # run regression
    X = add_constant(X)
    ols = OLS(y.values, X)
    ols = ols.fit()
    pred = ols.predict()
    resid = y - pred
    
    # organize results
    out = pd.DataFrame(ols.summary2().tables[1][0:1][['Coef.','t']])
    out = out.rename(columns={'Coef.':'alpha','t':'t-stat'}).reset_index(drop=True)
    out['bin'] = decile
    
    # get R-squared
    R2 = pd.Series(ols.summary2().tables[0][1][6])

    return resid, out, R2

In [7]:
# define function for high-low t-test
def high_low(df):
    
    # setup high-low
    high = df[df['bin'] == 10].reset_index(drop=True)
    high = high.merge(ff3,left_on=['year','month'],right_on=['year','month']).drop(columns=['date'])
    high['rb-rf'] = high['return'] - high['RF'] # get risk premium
    
    low = df[df['bin'] == 1].reset_index(drop=True)
    low = low.merge(ff3,left_on=['year','month'],right_on=['year','month']).drop(columns=['date'])
    low['rb-rf'] = low['return'] - low['RF'] # get risk premium
    
    # run regression
    X_high = high[['Mkt-RF','SMB','HML']]
    y_high = high['rb-rf']
    
    X_low = low[['Mkt-RF','SMB','HML']]
    y_low = low['rb-rf']
    
    # run regression
    X_high = add_constant(X_high)
    hi = OLS(y_high.values, X_high)
    hi = hi.fit()
    high_beta = pd.DataFrame(hi.summary2().tables[1][0:1][['Coef.','Std.Err.']])
    
    X_low = add_constant(X_low)
    lo = OLS(y_low.values, X_low)
    lo = lo.fit()
    low_beta = pd.DataFrame(lo.summary2().tables[1][0:1][['Coef.','Std.Err.']])
    
    # add betas
    beta = high_beta['Coef.'] - low_beta['Coef.']
    se = np.sqrt(high_beta['Std.Err.']**2 + low_beta['Std.Err.']**2)
    t = beta/se
    
    # organize results
    out = pd.DataFrame([beta, t])
    out = out.rename(index={'Coef.':'alpha','Unnamed 0':'t-stat'}, columns={'const':'High-Low'})
    
    return out.T

In [8]:
# Define F-test function
def F_test(alpha_hat, resids, F):
    
    # calulate formula inputs
    mu_bar = F.apply(np.mean) # sample means of factors
    T = resids.shape[0] # get periods T (months)
    N = alpha_hat.shape[0] # get portfolios (deciles)
    L = mu_bar.shape[0] # get factors count (MKRF, SML, HML)
    sigma_hat = resids.cov()*(T/(T-L-1)) # residual covar matrix
    omega_hat = F.cov()*(T/(T-1)) # factor covar matrix
    aSa = alpha_hat.T @ np.linalg.inv(sigma_hat) @ alpha_hat
    mOm = mu_bar.T @ np.linalg.inv(omega_hat) @ mu_bar
    
    # execute F-test
    F = (T/N)*((T-N-L)/(T-L-1))*(aSa/(1+mOm))
    
    # absolut mean of alphas
    alpha_mean = np.mean(abs(alpha_hat))
    
    # build output df
    F_table = pd.DataFrame([resids.shape[0],F,alpha_mean],index=['Months','F-Stat','Avg |alpha|']).T
    
    return F_table

In [9]:
# define main function
def run(start,end,step5=False):
    
    # define f_mat for lenght calcs
    F_mat = ff3[ff3['year'] >= start] # isolate start factor matrix
    F_mat = F_mat[F_mat['year'] < end].iloc[:,1:4] # complete factor matrix
    l = F_mat.shape[0]
    
    # run over years
    main = pd.DataFrame()
    for year in range(start,end):
        dec = rebalance(year,step5)
        main = main.append(dec).reset_index(drop=True)
        
    # run over deciles
    alphas = pd.DataFrame()
    resids = pd.DataFrame(np.zeros((l,10)),columns=list(range(1,11)))
    R2 = pd.DataFrame(dtype='float64')
    for decile in range(1,11):
        resid, out, r2 = tests(decile,main)
        alphas = alphas.append(out).reset_index(drop=True)
        resids[decile] = resid
        R2 = R2.append(r2, ignore_index=True).reset_index(drop=True)
        
    # run F-test
    alpha_hat = alphas['alpha'] # estimated intercepts
    F = F_test(alpha_hat, resids, F_mat)
    name = str(start) + '-' + str(end-1)
    F = F.rename(index={0:name})
    R2 = R2[0].map(float)
    F['Avg R-2'] = np.mean(R2.values)
    
    # get hgih-low t-test results
    out = high_low(main)    

    # adjust final alpha df
    alphas = alphas.set_index('bin')
    alphas = alphas.append(out)
    alphas = alphas.T
    
    return alphas, F, main

### Main Portfolio Run

In [10]:
# run main function
alphas_main, F_main, main = run(1967,2020)

# output F-stat
F_main

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,Months,F-Stat,Avg |alpha|,Avg R-2
1967-2019,636.0,2.416665,0.003151,0.4906


In [11]:
# output alphas w/ t-scores
alphas_main

,1,2,3,4,5,6,7,8,9,10,High-Low
alpha,-0.003613,-0.005995,-0.004505,-0.004907,0.001788,0.001166,-0.000768,0.002554,0.005308,0.000904,0.004516
t-stat,-1.163600,-2.873162,-1.792035,-2.545599,0.896008,0.645537,-0.348801,1.219788,2.348285,0.298110,1.040893


In [12]:
# isolate levels
main_year = (1+main.groupby(['bin','year'])[['return']].mean())**12 - 1
main_bin = main_year.groupby(['bin'])[['return']].mean()
main_bin['1$'] = (1+main_bin['return'])**(F_main.iloc[0,0]/12)
main_bin['return'] = main_bin['return'].map(lambda x: str(round(x*100,2)) + '%')
main_bin.T

bin,1,2,3,4,5,6,7,8,9,10
return,24.45%,11.02%,14.97%,11.42%,18.9%,17.76%,15.47%,22.01%,24.2%,23.64%
1$,108550,255.333,1626.73,307.751,9654.78,5795.3,2050.41,38001,97507.1,76725


### Step 5: Drop largest 1000

In [13]:
# step 5
# run main function
alphas_step5, F_step5, step5 = run(1967, 2020, step5=True)

# output F-stat and alphas w/ t-scores
F_step5

,Months,F-Stat,Avg |alpha|,Avg R-2
1967-2019,636.0,2.236237,0.005096,0.3386


In [14]:
# output alphas w/ t-scores
alphas_step5

,1,2,3,4,5,6,7,8,9,10,High-Low
alpha,-0.002859,-0.007092,-0.006882,-0.003633,0.003426,0.005016,0.002403,0.001776,0.011017,0.006858,0.009717
t-stat,-0.746059,-1.997839,-2.162732,-1.216029,1.060345,1.972684,0.779574,0.703641,1.974026,1.156501,1.376262


### Step 6: Compare first and second halves

In [15]:
# first half of data
# run main function
alphas_1, F_1, half_1 = run(1967, 1994)

# output alphas w/ t-scores
alphas_1

,1,2,3,4,5,6,7,8,9,10,High-Low
alpha,-0.003941,-0.004211,-0.007647,-0.003047,0.003355,0.000281,-0.000900,0.003072,0.005803,0.000341,0.004282
t-stat,-0.867174,-1.259392,-1.812291,-0.950574,1.024827,0.094916,-0.258722,0.976693,1.692616,0.067739,0.631336


In [16]:
# second half of data
# run main function
alphas_2, F_2, half_2 = run(1994, 2020)

# output alphas w/ t-scores
alphas_2

,1,2,3,4,5,6,7,8,9,10,High-Low
alpha,-0.002338,-0.007641,-0.001045,-0.005788,0.000585,0.001807,-0.001266,0.002109,0.003902,-0.001056,0.001282
t-stat,-0.567501,-3.162561,-0.441533,-2.844378,0.259103,0.925307,-0.478080,0.813491,1.361524,-0.335462,0.247303


In [17]:
# compare halves
F = F_1.append(F_2)
F = F.append(F_main)
F

,Months,F-Stat,Avg |alpha|,Avg R-2
1967-1993,324.0,1.034841,0.003260,0.4958
1994-2019,312.0,2.168211,0.002754,0.5382
1967-2019,636.0,2.416665,0.003151,0.4906
